In [21]:
#imports
import pymysql
import nltk
import os
import pandas as pd
import praw
import mysql.connector

from sqlalchemy import create_engine
from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

In [22]:
load_dotenv()

True

In [23]:
id = os.getenv("client_id")
secret = os.getenv("client_secret")
pw = os.getenv("pw")
user = "Scraper 1.0 by /u/ZaaLiah"

In [24]:
reddit = praw.Reddit(client_id=id, client_secret=secret, user_agent=user)

In [25]:
df = []
for submission in reddit.subreddit('politics').hot(limit=1000):
    df.append([submission.id, submission.created_utc, submission.title, submission.num_comments, submission.score])
df = pd.DataFrame(df, columns = ['id', 'timestamp', 'title', 'num_comments', 'score'])
df['date'] = pd.to_datetime(df['timestamp'], unit='s').dt.date


In [26]:
df = df.drop('timestamp', axis=1)
df = df[['id', 'date', 'title', 'num_comments', 'score']]

In [27]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = pw,
    database="localdb",
    port='3306'
  )

cursor = mydb.cursor()
for row in df.itertuples():
  cursor.execute(
    'INSERT INTO f_reddit_posts (id, date, title, num_comments, score)VALUES (%s,%s,%s,%s,%s) ON DUPLICATE KEY UPDATE id=VALUES (id)',
    (row.id,row.date,row.title,row.num_comments,row.score),
  row)
mydb.commit()
cursor.close()

True

In [28]:
sql = """
SELECT *
FROM f_reddit_posts
"""
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = pw,
    database="localdb",
    port='3306'
  )
df = pd.read_sql(sql, mydb)

c:\Users\bayli\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [29]:
sia = SIA()
df['compound'] = [sia.polarity_scores(x)['compound'] for x in df['title']]
df['neg'] = [sia.polarity_scores(x)['neg'] for x in df['title']]
df['neu'] = [sia.polarity_scores(x)['neu'] for x in df['title']]
df['pos'] = [sia.polarity_scores(x)['pos'] for x in df['title']]

In [30]:
df['label'] = 0
df.loc[df['compound'] > 0.2, 'label'] = 1
df.loc[df['compound'] < -0.2, 'label'] = -1
df = df[['id', 'compound', 'neg', 'neu', 'pos', 'label']]

In [31]:
cursor = mydb.cursor()
for row in df.itertuples():
  cursor.execute(
    'INSERT INTO d_rp_sentiment (id, compound, neg, neu, pos, label)VALUES (%s, %s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE id=VALUES (id)',
    (row.id,row.compound,row.neg,row.neu,row.pos,row.label),
  row)
mydb.commit()
cursor.close()

True